In [162]:
import numpy as np

import xml.etree.ElementTree as ElementTree

`triangle_area` evaluates the triangle area $A$ using Heron's formula:

$$ A = \sqrt{s (s-a) (s-b) (s-c)} \ , $$
where $s$ is the semi-perimeter, $a$ is the length of side a, $b$ is the length of side b, and $c$ is the length of side c.

In [163]:
def triangle_area(vertices):
    """
    Returns the triangle's area.

    :param vertices: the vertices of the triangle.
    :type vertices: two-dimensional array.
    :return: the triangle area.
    :return type: float.
    """
    a = np.linalg.norm(vertices[1] - vertices[0])
    b = np.linalg.norm(vertices[2] - vertices[1])
    c = np.linalg.norm(vertices[0] - vertices[2])
    semiperimeter = (a + b + c) * 0.5
    area = np.sqrt(semiperimeter * (semiperimeter - a) * (semiperimeter - b) * (semiperimeter - c))
    return area

`triangle_centroid` evaluates the centroid $C$ of the triangular face following

$$C = \frac{v_a + v_b + v_c}{3} \ , $$
where $v_a$, $v_b$, and $v_c$ are the triangle's vertices.

In [164]:
def triangle_centroid(vertices):
    """
    Returns the triangle's centroid.

    :param vertices: the vertices of the triangle.
    :type vertices: two-dimensional array.
    :return: the triangle's centroid.
    :return type: one-dimensional array with 3 columns.
    """
    centroid = (vertices[0] + vertices[1] + vertices[2]) / 3
    return centroid

`triangle_normal` evaluates the face normal of a triangle of vertices $v_a$, $v_b$, and $v_c$ following

$$n = (v_b - v_a) \times (v_c - v_a) \ , $$
where $\times$ represents a cross product.

In [165]:
def triangle_normal(vertices):
    """
    Returns the face normal of a triangle.

    :param vertices: the vertices of the triangle.
    :type vertices: two-dimensional array.
    :return: the triangle's normal.
    :return type: one-dimensional array with 3 columns.
    """
    side_x = vertices[1] - vertices[0]
    side_y = vertices[2] - vertices[0]
    normal = np.cross(side_x, side_y)
    return normal

`triangle_plane` evaluates the plane defined by the three vertices of a triangle.

In [166]:
def triangle_plane(vertices):
    """
    Returns the plane defined by vertices.

    :param vertices: the vertices of the triangle.
    :type vertices: two-dimensional array.
    :return: the triangle's plane.
    :return type: one-dimensional array with 4 columns.
    """
    pass

`check_visibility` verifies if a plane still allow the two faces to see each other.

In [167]:
def check_visibility(centroid_a, centroid_b, plane):
    """
    Returns true if the plane still allow the two faces to see each other.

    :param centroid_a: the centroid of face a.
    :type centroid_a: one-dimensional array with three columns.
    :param centroid_b: the centroid of face b.
    :type centroid_b: one-dimensional array with three columns.
    :param plane: the plane that may be between face a and face b.
    :type plane: one-dimensional array with 4 columns.
    :return: true if the two faces can see each other.
    :return type: bool.
    """
    pass

Testing the previous functions.

In [168]:
vertices_test = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]])
vertices_test_area = triangle_area(vertices_test)
vertices_test_centroid = triangle_centroid(vertices_test)
vertices_test_normal = triangle_normal(vertices_test)
assert(vertices_test_area - 0.5 < 1e-9)
assert(np.linalg.norm(vertices_test_centroid - np.array([1 / 3, 1 / 3, 0])) < 1e-9)
assert(np.linalg.norm(vertices_test_normal - np.array([0, 0, 1])) < 1e-9)

Classes definition.

In [169]:
class Solid(object):
    """
    Represents a 3D solid.
    """

    def __init__(self, faces, id):
        """
        Creates a new 3 dimensional solid.        
        
        :param faces: the triangular faces of the solid.
        :type faces: TriangularFace.
        :param id: the solid's id.
        :type id: string.
        """
        self.faces = faces
        self.id = id

In [193]:
class TriangularFace(object):
    """
    Represents a triangular face.
    """

    def __init__(self, vertices, colors):
        """
        Creates a new triangular face.

        :param vertices: the triangle's vertices.
        :type vertices: numpy array.
        :param colors: the color of each vertice.
        :type colors: numpy array.
        """
        self.vertices = vertices
        self.colors = colors
        self.area = triangle_area(self.vertices)
        self.centroid = triangle_centroid(self.vertices)
        self.normal = triangle_normal(self.vertices)

Parsing the COLLADA file with standard Python library to parse xml.

In [194]:
tree = ElementTree.parse('data/scene.dae')
root = tree.getroot()

Finding the library geometries section.

In [195]:
index = 0
for child in root:
    if "library_geometries" in child.tag:
        library_geometries_index = index
        break
    index += 1
library_geometries = root[library_geometries_index]

Create Solid objects based on COLLADA data.

In [196]:
solid_scene = []
for child in library_geometries:
    id = child.attrib["id"]
    mesh_positions = None
    mesh_colors = None
    mesh_material = None
    for child_geometry in child[0]:
        if "id" in list(child_geometry.attrib.keys()):
            if "mesh-positions" in child_geometry.attrib["id"]:
                mesh_positions = child_geometry
            elif "mesh-colors-Color" in child_geometry.attrib["id"]:
                mesh_colors = child_geometry
        elif "material" in list(child_geometry.attrib.keys()):
            mesh_material = child_geometry

    vertices_array = np.empty((0, 3), np.float64)
    if mesh_positions is not None:
        for child_mesh_positions in mesh_positions:
            if "id" in list(child_mesh_positions.attrib.keys()):
                if "mesh-positions-array" in child_mesh_positions.attrib["id"]:
                    values = child_mesh_positions.text.split(" ")
                    number_of_values = len(values)
                    number_of_vertices = number_of_values // 3
                    for i in range(number_of_vertices):
                        x_coordinate = float(values[i * 3])
                        y_coordinate = float(values[i * 3 + 1])
                        z_coordinate = float(values[i * 3 + 2])
                        vertices_array = np.append(vertices_array, np.array([[x_coordinate, y_coordinate, z_coordinate]]), axis=0)

    face_colors_array = np.empty((0, 4), np.float64)
    if mesh_colors is not None:
        for child_mesh_colors in mesh_colors:
            if "id" in list(child_mesh_colors.attrib.keys()):
                if "mesh-colors-Color-array" in child_mesh_colors.attrib["id"]:
                    values = child_mesh_colors.text.split(" ")
                    number_of_values = len(values)
                    number_of_faces = number_of_values // 12
                    for i in range(number_of_faces):
                        face_color = np.empty((0, 4), np.float64)
                        for j in range(3):
                            red = float(values[i * 4])
                            green = float(values[i * 4 + 1])
                            blue = float(values[i * 4 + 2])
                            alpha = float(values[i * 4 + 3])
                            face_color = np.append(face_color, np.array([[red, green, blue, alpha]]), axis=0)
                        face_colors_array = np.append(face_colors_array, face_color, axis=0)

    if mesh_material is not None:
        number_of_triangles = int(mesh_material.attrib["count"])
        step = 0
        for child_mesh_material in mesh_material:
            if len(list(child_mesh_material.keys())) == 0:
                faces_definition = child_mesh_material.text.split(" ")
                faces_list = []
                for i in range(number_of_triangles):
                    index_vertice_a = int(faces_definition[i * 3 * step])
                    index_vertice_b = int(faces_definition[i * 3 * step + step])
                    index_vertice_c = int(faces_definition[i * 3 * step + 2 * step])

                    color_a = None
                    color_b = None
                    color_c = None
                    if len(face_colors_array) != 0:
                        index_color_a = int(faces_definition[i * 3 * step + step - 1])
                        index_color_b = int(faces_definition[i * 3 * step + 2 * step - 1])
                        index_color_c = int(faces_definition[i * 3 * step + 3 * step - 1])

                        color_a = face_colors_array[index_color_a]
                        color_b = face_colors_array[index_color_b]
                        color_c = face_colors_array[index_color_c]

                    vertice_a = vertices_array[index_vertice_a]
                    vertice_b = vertices_array[index_vertice_b]
                    vertice_c = vertices_array[index_vertice_c]

                    vertices_face = np.array([vertice_a, vertice_b, vertice_c])
                    colors_face = np.array([[color_a], [color_b], [color_c]])
                    face = TriangularFace(vertices_face, colors_face)
                    faces_list.append(face)
            step += 1

        solid = Solid(faces_list, id)
        solid_scene.append(solid)

In [197]:
solid_scene[0].id

'Cube_002-mesh'